In [73]:
import pypstruct
import math

In [74]:
pdb = "graphene2by2.pdb"

In [75]:
#Load pdf file with a pdb parser
pdb_parsed = pypstruct.parseFilePDB(pdb)

#Pdb file is loaded in a big dictionnary
structure_dic = pdb_parsed.atomDictorize

print( structure_dic.keys())

for key in structure_dic:
    print( key, structure_dic[key][10])

dict_keys(['x', 'y', 'z', 'seqRes', 'chainID', 'resName', 'name'])
x -12.335
y -6.409
z 0.277
seqRes 2 
chainID G
resName GP0
name C1


In [76]:
structure_dic.keys()

dict_keys(['x', 'y', 'z', 'seqRes', 'chainID', 'resName', 'name'])

In [77]:
#Now we create a class "atom"
#With position, name and more ...

class atom:
    def __init__(self, x , y , z , seqRes ,chainID, resName, name ):
        self.x = x
        self.y = y
        self.z = z
        self.seqRes = seqRes
        self.chainID = chainID
        self.resName = resName
        self.name = name
        self.occupied = False

    def isCarbon(self):
        return self.name.startswith('C')

    def isHydrogen(self):
        return self.name.startswith('H')
    
    def __repr__ (self):
        return f"Atom(name={self.name}, res={self.seqRes}, x={self.x}, y={self.y}, occupied={self.occupied})"
    def __eq__(self, atom):
        return self.x == atom.x and self.y == atom.y and self.z == atom.z
    

In [78]:
#Instanciate atom object with the dictionnary content and add in a list

listAtoms = []
numberOfAtom = len(structure_dic["name"])

for i in range(0, numberOfAtom ) :
    
    listAtoms.append( atom( structure_dic["x"][i], 
                          structure_dic["y"][i],
                          structure_dic["z"][i],
                          structure_dic["seqRes"][i],
                          structure_dic["chainID"][i],
                          structure_dic["resName"][i],
                          structure_dic["name"][i] ) )


In [79]:
#subest hydrogen
atoms_h = [atom for atom in listAtoms if atom.isHydrogen()]
print( "Number of H :", len(atoms_h))

Number of H : 38


In [80]:
#subest carbon
atoms_c = [atom for atom in listAtoms if atom.isCarbon()]
print("Number of C :",  len(atoms_c))

Number of C : 198


In [81]:
# trouver les 4 lignes de bords # find the 4 edge lines

min_x = min(set([a.x for a in atoms_c]))
list_min_x = [a for a in atoms_c if a.x == min_x ]


max_x = max(set([a.x for a in atoms_c]))
list_max_x = [a for a in atoms_c if a.x == max_x ]

min_y = min(set([a.y for a in atoms_c]))
list_min_y = [a for a in atoms_c if a.y == min_y ]


max_y = max(set([a.y for a in atoms_c]))
list_max_y = [a for a in atoms_c if a.y == max_y ]


print("Number minX" ,len(list_min_x) ,"Number maxX" ,len(list_max_x) )
print("Number minY" ,len(list_min_y) ,"Number maxY" ,len(list_max_y) )

Number minX 8 Number maxX 10
Number minY 9 Number maxY 9


In [82]:
# Find the zigzag lines 
# in zig zag line every distance between carbon is the same 
#check distance the 3 first atom 

def dist( atom1 , atom2) :
    return round(math.sqrt( math.pow(atom1.x-atom2.x , 2) +  math.pow(atom1.y-atom2.y, 2) +  math.pow(atom1.z-atom2.z,2) ), 2)


def isDistanceEqual( li ):
    dist_1_2 = dist(li[0] , li[1])
    dist_2_3 = dist(li[1] , li[2])
    return dist_1_2 == dist_2_3

# Sort each atom according to their coordinates (x or y) 
def sortListAtom(li):
    if li[0].x == li[1].x : 
        return sorted(li, key=lambda atom: atom.y)
    else:
        return sorted(li, key=lambda atom: atom.x)

# We have the edges, we need to define which one are armchair and zigzag.
# In zigzag, distance between all carbons are the same and not in armchair.   

for l in [list_min_x, list_min_y]:
    if(isDistanceEqual(sortListAtom(l))):
        zigzagBottom = l
    else:
        chairBottom = l

for l in [list_max_x, list_max_y]:
    if(isDistanceEqual(sortListAtom(l))):
        zigzagTop = l
    else:
        chairTop = l

print(len(zigzagBottom), "carbons in zigzag bottom")    
print(len(zigzagTop), "carbons in zigzag top") 
print(len(chairBottom), "carbons in armchair bottom") 
print(len(chairTop), "carbons in armchair top") 

9 carbons in zigzag bottom
9 carbons in zigzag top
8 carbons in armchair bottom
10 carbons in armchair top


### Define research function

In [83]:
def findClosest(atom, listAtom):
    # return a list of the closest free atom (min 1 element and maximum X elements)
    #You can find the closest atom depending of the list that you give (only carbon or only hydrogene or both)
    
    start = 0 
    #print("###", atom)
    #Find the first inoccupied carbon
    for atom_c in listAtom:
        if not atom_c.occupied and not atom == atom_c: 
            
            min_distance = dist(atom, atom_c)
            #print(atom_c, min_distance)
            #( atom , distance from c)
            closest = [ atom_c ]
            break
        start+=1
    
    for atom_c in listAtom[start+1:]:
        
        if not atom_c.occupied and not atom == atom_c: 
            current_dist = dist(atom, atom_c)
            #if current_dist < 5:
                #print(atom_c, current_dist)
            if current_dist < min_distance:
                min_distance = current_dist
                closest = [ atom_c ]
                
            elif current_dist == min_distance:
                closest.append( atom_c)
             
    
    return closest

### Fill the armchair

In [84]:
for atom in chairBottom:
    print(atom, findClosest(atom, atoms_h))

Atom(name=C8, res=1 , x=-12.335, y=-7.834, occupied=False) [Atom(name=H8, res=1 , x=-12.952, y=-8.191, occupied=False)]
Atom(name=C1, res=2 , x=-12.335, y=-6.409, occupied=False) [Atom(name=H1, res=2 , x=-12.952, y=-6.053, occupied=False)]
Atom(name=C8, res=2 , x=-12.335, y=-3.561, occupied=False) [Atom(name=H8, res=2 , x=-12.952, y=-3.918, occupied=False)]
Atom(name=C1, res=3 , x=-12.335, y=-2.136, occupied=False) [Atom(name=H1, res=3 , x=-12.952, y=-1.78, occupied=False)]
Atom(name=C8, res=3 , x=-12.335, y=0.711, occupied=False) [Atom(name=H8, res=3 , x=-12.952, y=0.355, occupied=False)]
Atom(name=C1, res=4 , x=-12.335, y=2.137, occupied=False) [Atom(name=H1, res=4 , x=-12.952, y=2.493, occupied=False)]
Atom(name=C8, res=4 , x=-12.335, y=4.985, occupied=False) [Atom(name=H8, res=4 , x=-12.952, y=4.628, occupied=False)]
Atom(name=C1, res=5 , x=-12.335, y=6.409, occupied=False) [Atom(name=H1, res=5 , x=-12.952, y=6.766, occupied=False)]


In [85]:
# We take each pair of carbon and add the closest hydrogen to each to have a group of 4 atoms
def groupArmchair(listC, listH):
    step = 2
    armchair_group = []
    group = []
    for atom_c in listC:
        group.append(atom_c)
        #This atom is not free anymore
        atom_c.occupied = True
        
        closest_atoms_h = findClosest(atom_c, listH)
        if len(closest_atoms_h) == 1:
            atom_h = closest_atoms_h[0]
            atom_h.occupied = True
            group.append(atom_h)
            step -= 1 
            if step == 0:
                armchair_group.append(group)
                group = []
                step = 2
        else :
            print( "WTF" , closest_atoms_h)
    return armchair_group
    

In [86]:
#We add armchair beads to list of all beads
listBeads = []
listBeads += groupArmchair(chairBottom, atoms_h)
listBeads += groupArmchair(chairTop, atoms_h)

listBeads

[[Atom(name=C8, res=1 , x=-12.335, y=-7.834, occupied=True),
  Atom(name=H8, res=1 , x=-12.952, y=-8.191, occupied=True),
  Atom(name=C1, res=2 , x=-12.335, y=-6.409, occupied=True),
  Atom(name=H1, res=2 , x=-12.952, y=-6.053, occupied=True)],
 [Atom(name=C8, res=2 , x=-12.335, y=-3.561, occupied=True),
  Atom(name=H8, res=2 , x=-12.952, y=-3.918, occupied=True),
  Atom(name=C1, res=3 , x=-12.335, y=-2.136, occupied=True),
  Atom(name=H1, res=3 , x=-12.952, y=-1.78, occupied=True)],
 [Atom(name=C8, res=3 , x=-12.335, y=0.711, occupied=True),
  Atom(name=H8, res=3 , x=-12.952, y=0.355, occupied=True),
  Atom(name=C1, res=4 , x=-12.335, y=2.137, occupied=True),
  Atom(name=H1, res=4 , x=-12.952, y=2.493, occupied=True)],
 [Atom(name=C8, res=4 , x=-12.335, y=4.985, occupied=True),
  Atom(name=H8, res=4 , x=-12.952, y=4.628, occupied=True),
  Atom(name=C1, res=5 , x=-12.335, y=6.409, occupied=True),
  Atom(name=H1, res=5 , x=-12.952, y=6.766, occupied=True)],
 [Atom(name=C4, res=21 , x=11

## Fill the zigzag

In [87]:
#For the zig zag part, beads are compposed by 
# 1 H and 2 C
# and for the last beads 2H and 2C


print(zigzagBottom)

def groupZigzag(atomsZigzag, listCarbons, listHydrogens):
    zigzag_group = []
    
    for atom in atomsZigzag:
        bead = []
        
        closestC = findClosest( atom, listCarbons )
        closestH = findClosest( atom, listHydrogens )
        
        #Now we need to check if closest Carbon(s) are bind to an hydrogen 
        #If they are -> beads 2C2H
        
        
        #try to find the type of beads H-C-C-H 
        for c in closestC :
            for x in findClosest( c, listAtoms ):
                if x.isHydrogen() :
                    bead = [atom, closestH[0], c, x] 
                    
        if bead == []:
            bead = [atom, closestH[0], closestC[0]]
              
        for b in bead:
            b.occupied = True
        zigzag_group.append(bead)
    #si c'est une beads normal de 1h and 2c 
    
    #si c'est une beads de 2c et 2h faire la verif
    
    #init list of atoms, each element is a list of atoms who are in one bedas

    return zigzag_group
    

[Atom(name=C3, res=1 , x=-9.868, y=-10.682, occupied=False), Atom(name=C1, res=6 , x=-7.401, y=-10.682, occupied=False), Atom(name=C3, res=6 , x=-4.934, y=-10.682, occupied=False), Atom(name=C1, res=11 , x=-2.467, y=-10.682, occupied=False), Atom(name=C3, res=11 , x=0.0, y=-10.682, occupied=False), Atom(name=C1, res=16 , x=2.467, y=-10.682, occupied=False), Atom(name=C3, res=16 , x=4.934, y=-10.682, occupied=False), Atom(name=C1, res=21 , x=7.401, y=-10.682, occupied=False), Atom(name=C3, res=21 , x=9.868, y=-10.682, occupied=False)]


In [88]:
zigzagBottom_group = groupZigzag(zigzagBottom, atoms_c, atoms_h)
listBeads.append( zigzagBottom_group)
zigzagBottom_group

[[Atom(name=C3, res=1 , x=-9.868, y=-10.682, occupied=True),
  Atom(name=H3, res=1 , x=-9.869, y=-11.394, occupied=True),
  Atom(name=C2, res=1 , x=-11.101, y=-9.97, occupied=True),
  Atom(name=H2, res=1 , x=-11.718, y=-10.326, occupied=True)],
 [Atom(name=C1, res=6 , x=-7.401, y=-10.682, occupied=True),
  Atom(name=H1, res=6 , x=-7.402, y=-11.394, occupied=True),
  Atom(name=C4, res=1 , x=-8.634, y=-9.97, occupied=True)],
 [Atom(name=C3, res=6 , x=-4.934, y=-10.682, occupied=True),
  Atom(name=H3, res=6 , x=-4.935, y=-11.394, occupied=True),
  Atom(name=C2, res=6 , x=-6.167, y=-9.97, occupied=True)],
 [Atom(name=C1, res=11 , x=-2.467, y=-10.682, occupied=True),
  Atom(name=H1, res=11 , x=-2.468, y=-11.394, occupied=True),
  Atom(name=C4, res=6 , x=-3.7, y=-9.97, occupied=True)],
 [Atom(name=C3, res=11 , x=0.0, y=-10.682, occupied=True),
  Atom(name=H3, res=11 , x=-0.001, y=-11.394, occupied=True),
  Atom(name=C2, res=11 , x=-1.233, y=-9.97, occupied=True)],
 [Atom(name=C1, res=16 , x=

In [89]:
zigzagTop_group =groupZigzag(zigzagTop, atoms_c, atoms_h)
listBeads.append( zigzagTop_group)
zigzagTop_group

[[Atom(name=C6, res=5 , x=-9.868, y=9.257, occupied=True),
  Atom(name=H6, res=5 , x=-9.869, y=9.97, occupied=True),
  Atom(name=C7, res=5 , x=-11.101, y=8.545, occupied=True),
  Atom(name=H7, res=5 , x=-11.718, y=8.902, occupied=True)],
 [Atom(name=C6, res=10 , x=-4.934, y=9.257, occupied=True),
  Atom(name=H6, res=10 , x=-4.935, y=9.97, occupied=True),
  Atom(name=C5, res=10 , x=-3.7, y=8.545, occupied=True)],
 [Atom(name=C8, res=10 , x=-7.401, y=9.257, occupied=True),
  Atom(name=H8, res=10 , x=-7.402, y=9.97, occupied=True),
  Atom(name=C5, res=5 , x=-8.634, y=8.545, occupied=True)],
 [Atom(name=C6, res=15 , x=0.0, y=9.257, occupied=True),
  Atom(name=H6, res=15 , x=-0.001, y=9.97, occupied=True),
  Atom(name=C5, res=15 , x=1.234, y=8.545, occupied=True)],
 [Atom(name=C8, res=15 , x=-2.467, y=9.257, occupied=True),
  Atom(name=H8, res=15 , x=-2.468, y=9.97, occupied=True),
  Atom(name=C7, res=15 , x=-1.233, y=8.545, occupied=True)],
 [Atom(name=C6, res=20 , x=4.934, y=9.257, occupi

In [90]:
listBeads

[[Atom(name=C8, res=1 , x=-12.335, y=-7.834, occupied=True),
  Atom(name=H8, res=1 , x=-12.952, y=-8.191, occupied=True),
  Atom(name=C1, res=2 , x=-12.335, y=-6.409, occupied=True),
  Atom(name=H1, res=2 , x=-12.952, y=-6.053, occupied=True)],
 [Atom(name=C8, res=2 , x=-12.335, y=-3.561, occupied=True),
  Atom(name=H8, res=2 , x=-12.952, y=-3.918, occupied=True),
  Atom(name=C1, res=3 , x=-12.335, y=-2.136, occupied=True),
  Atom(name=H1, res=3 , x=-12.952, y=-1.78, occupied=True)],
 [Atom(name=C8, res=3 , x=-12.335, y=0.711, occupied=True),
  Atom(name=H8, res=3 , x=-12.952, y=0.355, occupied=True),
  Atom(name=C1, res=4 , x=-12.335, y=2.137, occupied=True),
  Atom(name=H1, res=4 , x=-12.952, y=2.493, occupied=True)],
 [Atom(name=C8, res=4 , x=-12.335, y=4.985, occupied=True),
  Atom(name=H8, res=4 , x=-12.952, y=4.628, occupied=True),
  Atom(name=C1, res=5 , x=-12.335, y=6.409, occupied=True),
  Atom(name=H1, res=5 , x=-12.952, y=6.766, occupied=True)],
 [Atom(name=C4, res=21 , x=11

In [91]:
print(len(listBeads))

11


In [92]:
len(zigzagTop_group)

9

In [93]:
len(zigzagBottom_group)

9

In [94]:
zigzagBottom_group[0]

[Atom(name=C3, res=1 , x=-9.868, y=-10.682, occupied=True),
 Atom(name=H3, res=1 , x=-9.869, y=-11.394, occupied=True),
 Atom(name=C2, res=1 , x=-11.101, y=-9.97, occupied=True),
 Atom(name=H2, res=1 , x=-11.718, y=-10.326, occupied=True)]

In [95]:
zigzagBottom_group[1]

[Atom(name=C1, res=6 , x=-7.401, y=-10.682, occupied=True),
 Atom(name=H1, res=6 , x=-7.402, y=-11.394, occupied=True),
 Atom(name=C4, res=1 , x=-8.634, y=-9.97, occupied=True)]

In [96]:
zigzagBottom_group[-1]

[Atom(name=C3, res=21 , x=9.868, y=-10.682, occupied=True),
 Atom(name=H3, res=21 , x=9.867, y=-11.394, occupied=True),
 Atom(name=C2, res=21 , x=8.635, y=-9.97, occupied=True)]

In [97]:
zigzagTop_group[-1]

[Atom(name=C8, res=25 , x=7.401, y=9.257, occupied=True),
 Atom(name=H8, res=25 , x=7.4, y=9.97, occupied=True),
 Atom(name=C4, res=20 , x=6.168, y=7.121, occupied=True)]

In [98]:
zigzagTop_group[0]

[Atom(name=C6, res=5 , x=-9.868, y=9.257, occupied=True),
 Atom(name=H6, res=5 , x=-9.869, y=9.97, occupied=True),
 Atom(name=C7, res=5 , x=-11.101, y=8.545, occupied=True),
 Atom(name=H7, res=5 , x=-11.718, y=8.902, occupied=True)]

In [99]:
zigzagTop_group

[[Atom(name=C6, res=5 , x=-9.868, y=9.257, occupied=True),
  Atom(name=H6, res=5 , x=-9.869, y=9.97, occupied=True),
  Atom(name=C7, res=5 , x=-11.101, y=8.545, occupied=True),
  Atom(name=H7, res=5 , x=-11.718, y=8.902, occupied=True)],
 [Atom(name=C6, res=10 , x=-4.934, y=9.257, occupied=True),
  Atom(name=H6, res=10 , x=-4.935, y=9.97, occupied=True),
  Atom(name=C5, res=10 , x=-3.7, y=8.545, occupied=True)],
 [Atom(name=C8, res=10 , x=-7.401, y=9.257, occupied=True),
  Atom(name=H8, res=10 , x=-7.402, y=9.97, occupied=True),
  Atom(name=C5, res=5 , x=-8.634, y=8.545, occupied=True)],
 [Atom(name=C6, res=15 , x=0.0, y=9.257, occupied=True),
  Atom(name=H6, res=15 , x=-0.001, y=9.97, occupied=True),
  Atom(name=C5, res=15 , x=1.234, y=8.545, occupied=True)],
 [Atom(name=C8, res=15 , x=-2.467, y=9.257, occupied=True),
  Atom(name=H8, res=15 , x=-2.468, y=9.97, occupied=True),
  Atom(name=C7, res=15 , x=-1.233, y=8.545, occupied=True)],
 [Atom(name=C6, res=20 , x=4.934, y=9.257, occupi

In [100]:
zigzagTop_group[0][0].x

-9.868

In [101]:
import numpy as np

In [102]:
coords = []
for i in zigzagTop_group:
    
    x = []
    y = []
    for j in range(len(i)):
        x.append(i[j].x)
        y.append(i[j].y)
    coords.append((sum(x)/len(x), sum(y)/len(y), 0.280))

In [103]:
coords

[(-10.639, 9.1685, 0.28),
 (-4.523, 9.257333333333333, 0.28),
 (-7.812333333333334, 9.257333333333333, 0.28),
 (0.41100000000000003, 9.257333333333333, 0.28),
 (-2.0560000000000005, 9.257333333333333, 0.28),
 (5.345, 9.257333333333333, 0.28),
 (2.878, 9.257333333333333, 0.28),
 (9.456666666666665, 9.257333333333333, 0.28),
 (6.989666666666667, 8.782666666666666, 0.28)]

In [104]:
sorted(coords, key = lambda x: x[0])

[(-10.639, 9.1685, 0.28),
 (-7.812333333333334, 9.257333333333333, 0.28),
 (-4.523, 9.257333333333333, 0.28),
 (-2.0560000000000005, 9.257333333333333, 0.28),
 (0.41100000000000003, 9.257333333333333, 0.28),
 (2.878, 9.257333333333333, 0.28),
 (5.345, 9.257333333333333, 0.28),
 (6.989666666666667, 8.782666666666666, 0.28),
 (9.456666666666665, 9.257333333333333, 0.28)]

In [105]:
coords = []
for i in zigzagBottom_group:
    x = []
    y = []
    for j in range(len(i)):
        x.append(i[j].x)
        y.append(i[j].y)
    coords.append((sum(x)/len(x), sum(y)/len(y), 0.280))
    

In [106]:
coords

[(-10.639, -10.593, 0.28),
 (-7.812333333333334, -10.682, 0.28),
 (-5.3453333333333335, -10.682, 0.28),
 (-2.878333333333334, -10.682, 0.28),
 (-0.41133333333333333, -10.682, 0.28),
 (2.0556666666666668, -10.682, 0.28),
 (4.522666666666667, -10.682, 0.28),
 (6.989666666666667, -10.682, 0.28),
 (9.456666666666665, -10.682, 0.28)]

In [107]:
sorted(coords, key = lambda x: x[0])

[(-10.639, -10.593, 0.28),
 (-7.812333333333334, -10.682, 0.28),
 (-5.3453333333333335, -10.682, 0.28),
 (-2.878333333333334, -10.682, 0.28),
 (-0.41133333333333333, -10.682, 0.28),
 (2.0556666666666668, -10.682, 0.28),
 (4.522666666666667, -10.682, 0.28),
 (6.989666666666667, -10.682, 0.28),
 (9.456666666666665, -10.682, 0.28)]

In [108]:
chairBottom

[Atom(name=C8, res=1 , x=-12.335, y=-7.834, occupied=True),
 Atom(name=C1, res=2 , x=-12.335, y=-6.409, occupied=True),
 Atom(name=C8, res=2 , x=-12.335, y=-3.561, occupied=True),
 Atom(name=C1, res=3 , x=-12.335, y=-2.136, occupied=True),
 Atom(name=C8, res=3 , x=-12.335, y=0.711, occupied=True),
 Atom(name=C1, res=4 , x=-12.335, y=2.137, occupied=True),
 Atom(name=C8, res=4 , x=-12.335, y=4.985, occupied=True),
 Atom(name=C1, res=5 , x=-12.335, y=6.409, occupied=True)]

In [111]:
len(listBeads)

11

In [116]:
len(listBeads[10])

9

In [117]:
len(listBeads[9])

9

In [122]:
listBeads[2]

[Atom(name=C8, res=3 , x=-12.335, y=0.711, occupied=True),
 Atom(name=H8, res=3 , x=-12.952, y=0.355, occupied=True),
 Atom(name=C1, res=4 , x=-12.335, y=2.137, occupied=True),
 Atom(name=H1, res=4 , x=-12.952, y=2.493, occupied=True)]

In [123]:
for i in listBeads[:9]:
    print(len(i))

4
4
4
4
4
4
4
4
4


In [124]:
coords = []
for i in listBeads[:9]:
    x = []
    y = []
    for j in range(len(i)):
        x.append(i[j].x)
        y.append(i[j].y)
    coords.append((sum(x)/len(x), sum(y)/len(y), 0.280))
    

In [125]:
coords

[(-12.6435, -7.12175, 0.28),
 (-12.6435, -2.84875, 0.28),
 (-12.6435, 1.424, 0.28),
 (-12.6435, 5.696999999999999, 0.28),
 (11.4105, -9.25825, 0.28),
 (11.4105, -4.984999999999999, 0.28),
 (11.4105, -0.7125, 0.28),
 (11.4105, 3.561, 0.28),
 (11.4105, 7.833500000000001, 0.28)]

In [126]:
sorted(coords, key = lambda x: x[1])

[(11.4105, -9.25825, 0.28),
 (-12.6435, -7.12175, 0.28),
 (11.4105, -4.984999999999999, 0.28),
 (-12.6435, -2.84875, 0.28),
 (11.4105, -0.7125, 0.28),
 (-12.6435, 1.424, 0.28),
 (11.4105, 3.561, 0.28),
 (-12.6435, 5.696999999999999, 0.28),
 (11.4105, 7.833500000000001, 0.28)]